In [1]:
import json
import pandas as pd
from konlpy.tag import Okt # komoran, han, kkma
from jamo import h2j, j2hcj
import re

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

from tqdm.notebook import tqdm
import numpy as np

# 데이터 가져오기, 학습 데이터셋으로 만들기

## 인벤에서 수집한 데이터 가져오기

In [161]:
immoral = pd.read_csv(r'data\immoral.txt', sep = 'asssssssss', header = None)
immoral.columns = ['text']
immoral['immoral'] = True

C:\Users\user\AppData\Local\Temp\ipykernel_23136\2151298477.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  immoral = pd.read_csv(r'data\immoral.txt', sep = 'asssssssss', header = None)


In [162]:
clean = pd.read_csv(r'data\clean.txt', sep = 'asssssssss', header = None)
clean.columns = ['text']
clean['immoral'] = False

C:\Users\user\AppData\Local\Temp\ipykernel_23136\2827705178.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  clean = pd.read_csv(r'data\clean.txt', sep = 'asssssssss', header = None)


## op.gg 데이터 크롤링

In [31]:
import requests
from bs4 import BeautifulSoup
import json
import time

In [ ]:
lii = []
headers = {
    'Referer':'https://www.op.gg/',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
}
for chm in champ:
    n = 1
    while 1:
        url = f'https://op.gg/api/v1.0/internal/bypass/champions/{chm}/comments?&sort=popular&page={n}&limit=100&hl=ko_KR&is_latest_version=false'
        n += 1
        resp = requests.get(url, headers = headers)
        dict_ch = json.loads(resp.text)
        if len(dict_ch['data']) == 0:
            break
        
        for dat in dict_ch['data']:
            lii.append(dat['content'].replace('\n', ''))
        time.sleep(5)
    
    print(chm)

In [75]:
df_lii = pd.DataFrame(lii, columns = ['text'])
df_lii['immoral'] = ''
df_lii

,text,immoral
0,얘 와맆에 곰돌이가 에어본 시키는 거 생겼던데 여기에도 생기나,
1,자기고향 찾아오네,
2,몇달만의 고향이냐,
3,점멸 있을 때 한정 1티어,
4,스킬 사거리 표시 없이 하면 암걸리는 챔,
...,...,...
7968,의외로 꼴짤이 별로 없음,
7969,나피리 근데 초반 라인전도 약하고 카운터도 많음.그리고 돌진기가 다른 암살자와 다르...,
7970,0/6/0 야스오:야이 개쌔꺄 이게 챔이냐 야발4/0/2 나피리:개 맞는데0/8/0...,
7971,얘 한테 지면 사람아닌래끼다버프좀 하고 성능좀 버프히라못 쓸 정도다,


### op.gg 데이터 직접 라벨링, 라벨링 된 csv 가져오기

In [164]:
# df_lii.to_csv(r'data/champions.csv', index = False)
df_lii = pd.read_csv(r'data/champions.csv')

In [195]:
champions = pd.read_csv(r'data\champions.csv')
champions.columns = ['text','immoral']
champions.loc[champions['immoral'] == 0, 'immoral'] = False
champions.loc[champions['immoral'] == 1, 'immoral'] = True
champions = champions.fillna(False)
champions

C:\Users\user\AppData\Local\Temp\ipykernel_10380\2204739627.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  champions.loc[champions['immoral'] == 0, 'immoral'] = False


,text,immoral
0,얘 와맆에 곰돌이가 에어본 시키는 거 생겼던데 여기에도 생기나,False
1,자기고향 찾아오네,False
2,몇달만의 고향이냐,False
3,점멸 있을 때 한정 1티어,False
4,스킬 사거리 표시 없이 하면 암걸리는 챔,False
...,...,...
6458,클레드로 굶드라랑 월식 못 가게 너프먹는다니까 진짜 ㅈ같아서 겜 못해먹겠네 월식은 ...,True
6459,진짜 개사기임아니 협곡말고 롤체 학살자 클레드요,False
6460,진짜 태불방 롤백 시킬거 아니면 클레드는 건들면 안되지 ㅋㅋㅋ 선넘네 진짜,False
6461,"""바텀에서 겨우 더블킬을 했다.""""나팔소리가 들렸다.""""여름이었다.""(제압되었습니다!)",False


## Ai hub 데이터 가져오기

In [154]:
# ai hub train set
li = []
for i in range(1, 6):
    with open(r'.\data\train\labled\talksets-train-' + str(i) + '.json') as f:
        data = json.load(f)
    for d in data:
        for s in d['sentences']:
            li.append([s['origin_text'], s['is_immoral'], s['intensity'], 
                    'CENSURE' in s['types'], 'HATE' in s['types'], 'DISCRIMINATION' in s['types'], 
                    'SEXUAL' in s['types'], 'ABUSE' in s['types'], 'VIOLENCE' in s['types'], 'CRIME' in s['types']])
            
train = pd.DataFrame(li, columns=['text', 'immoral', 'intensity', 'CENSURE', 'HATE', 'DISCRIMINATION', 'SEXUAL', 'ABUSE', 'VIOLENCE', 'CRIME'])
train['text'] = train['text'].apply(lambda x : re.sub('#@.+?#', '', x.replace('#@#', '')))

train['immoral'] = (train['SEXUAL'] == True) | (train['ABUSE'] == True)
train_set = train[['text', 'immoral']]

In [156]:
# ai hub test set
li = []
with open(r'.\data\valid\labled\talksets-train-6.json') as f:
    data = json.load(f)
for d in data:
    for s in d['sentences']:
        li.append([s['origin_text'], s['is_immoral'], s['intensity'], 
                'CENSURE' in s['types'], 'HATE' in s['types'], 'DISCRIMINATION' in s['types'], 
                'SEXUAL' in s['types'], 'ABUSE' in s['types'], 'VIOLENCE' in s['types'], 'CRIME' in s['types']])
test = pd.DataFrame(li, columns=['text', 'immoral', 'intensity', 'CENSURE', 'HATE', 'DISCRIMINATION', 'SEXUAL', 'ABUSE', 'VIOLENCE', 'CRIME'])

test['text'] = test['text'].apply(lambda x : re.sub('#@.+?#', '', x.replace('#@#', '')))

test['immoral'] = (test['SEXUAL'] == True) | (test['ABUSE'] == True)
test_set = test[['text', 'immoral']]

## badwords 가져오기
https://github.com/organization/Gentleman/blob/master/resources/badwords.json

In [184]:
# badwords, train
with open(r".\data\train\labled\badwords.json") as f:
    data = json.load(f)

badwords = pd.DataFrame(data['badwords'], columns= ['text'])
badwords['immoral'] = True

## 가져온 데이터 train/test 나눠서 합치기

In [166]:
im_sl = int(len(immoral) * 0.8)
cl_sl = int(len(clean) * 0.8)
cp_sl = int(len(champions) * 0.8)
bw_sl = int(len(badwords) * 0.8)

In [211]:
train = pd.concat([train_set, immoral[:im_sl], clean[:cl_sl], badwords[:bw_sl], champions[:cp_sl]])
test = pd.concat([test_set, immoral[im_sl:], clean[cl_sl:], badwords[bw_sl:], champions[cp_sl:]])
train['immoral'] = train['immoral'].apply(lambda x : 1 if x == True else 0)
test['immoral'] = test['immoral'].apply(lambda x : 1 if x == True else 0)
train.shape, test.shape

((369610, 2), (46830, 2))

# 데이터 전처리


In [116]:
# train.to_csv(r'data/train_final.csv', index = False)
# test.to_csv(r'data/test_final.csv', index = False)
train = pd.read_csv(r'data/train_final.csv')
test = pd.read_csv(r'data/test_final.csv')
# train = pd.read_csv(r'data/train_aug.csv')
# test = pd.read_csv(r'data/test_aug.csv')

# test = pd.read_csv(r'data/champions_t.csv')
train = train.dropna()
test = test.dropna()
train.shape, test.shape

((369609, 2), (46830, 2))

In [169]:
test[test['immoral'] != 1]

,text,immoral
0,ㅇㅒ ㅇㅘㄹㅣㅍㅇㅔ ㄱㅗㅁㄷㅗㄹㅇㅣㄱㅏ ㅇㅔㅇㅓㅂㅗㄴ ㅅㅣㅋㅣㄴㅡㄴ ㄱㅓ ㅅㅐㅇㄱㅕ...,0.0
1,ㅈㅏㄱㅣㄱㅗㅎㅑㅇ ㅊㅏㅈㅇㅏㅇㅗㄴㅔ,0.0
2,ㅁㅕㅊㄷㅏㄹㅁㅏㄴㅇㅢ ㄱㅗㅎㅑㅇㅇㅣㄴㅑ,0.0
3,ㅈㅓㅁㅁㅕㄹ ㅇㅣㅆㅇㅡㄹ ㄸㅐ ㅎㅏㄴㅈㅓㅇ 1ㅌㅣㅇㅓ,0.0
4,ㅅㅡㅋㅣㄹ ㅅㅏㄱㅓㄹㅣ ㅍㅛㅅㅣ ㅇㅓㅄㅇㅣ ㅎㅏㅁㅕㄴ ㅇㅏㅁㄱㅓㄹㄹㅣㄴㅡㄴ ㅊㅐㅁ,0.0
...,...,...
6457,ㄹㅏㅇㅣㄴㅈㅓㄴㅇㅔ ㄴㅏㄹㅁㅓㄱㅎㅐㅅㅓ ㅎㅏㄴ 6ㅋㅣㄹ ㄷㅙㄷㅗ ㅃㅏㄹㄹㅣ ㅁㅗㅅㄲ...,0.0
6459,ㅈㅣㄴㅉㅏ ㄱㅐㅅㅏㄱㅣㅇㅣㅁㅇㅏㄴㅣ ㅎㅕㅂㄱㅗㄱㅁㅏㄹㄱㅗ ㄹㅗㄹㅊㅔ ㅎㅏㄱㅅㅏㄹㅈㅏ...,0.0
6460,ㅈㅣㄴㅉㅏ ㅌㅐㅂㅜㄹㅂㅏㅇ ㄹㅗㄹㅂㅐㄱ ㅅㅣㅋㅣㄹㄱㅓ ㅇㅏㄴㅣㅁㅕㄴ ㅋㅡㄹㄹㅔㄷㅡㄴ...,0.0
6461,"""ㅂㅏㅌㅓㅁㅇㅔㅅㅓ ㄱㅕㅇㅜ ㄷㅓㅂㅡㄹㅋㅣㄹㅇㅡㄹ ㅎㅐㅆㄷㅏ.""""ㄴㅏㅍㅏㄹㅅㅗㄹㅣㄱ...",0.0


## 데이터 자모음으로 쪼개기

In [117]:
train['text'] = train['text'].apply(lambda x : j2hcj(h2j(x)))
test['text'] = test['text'].apply(lambda x : j2hcj(h2j(x)))

In [118]:
# swap rows
train = train.sample(frac = 1).reset_index(drop=True)

In [119]:
def textToList(text):
    li = []
    for t in text:
        li.append(list(t.strip()))
    return li

In [120]:
# 자모음 쪼갠 텍스트를 한글자씩 리스트로 만들어서 데이터셋 만든다
def textToList(text):
    li = []
    for t in text:
        li.append(list(t))
    return li

train_list = textToList(train['text'])
train_label = train['immoral']

test_list = textToList(test['text'])
test_label = test['immoral']

# X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [121]:
# 문장 최대 길이 정해야되니까 문장 길이 분포가 얼마나 되어있는지 확인한다
# li_len = []
# for t in train_list:
#     li_len.append(len(t))
# import seaborn as sns
# sns.set_theme(style="darkgrid")
# sns.histplot(li_len)
## 100으로 자르면 문장 잘리는 아이들 비율이 2%

In [122]:
tokenizer = Tokenizer(num_words=110)
tokenizer.fit_on_texts(train_list) # 단어 인덱스 구축
text_sequences = tokenizer.texts_to_sequences(train_list) # 문자열 -> 인덱스 리스트
                                                            # '나는 천재다 나는 멋있다' -> [1, 2, 1, 3]
# train data로 fit 시켜준 tokenizer로 test data도 인덱스로 변경
text_sequences_test = tokenizer.texts_to_sequences(test_list)

word_vocab = tokenizer.word_index # 딕셔너리 형태
print("전체 단어 개수: ", len(word_vocab)) # 전체 단어 개수 확인

전체 단어 개수:  166


In [123]:
MAX_SEQUENCE_LENGTH = 100 # 문장 최대 길이

X_train = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 100 글자가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_train = np.array(train_label) # 각 리뷰의 감정을 넘파이 배열로 만든다.

X_test = pad_sequences(text_sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 100 글자가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_test = np.array(test_label) # 각 리뷰의 감정을 넘파이 배열로 만든다.

print('Shape of train input data tensor:', X_train.shape) # 리뷰 데이터의 형태 확인
print('Shape of train label tensor:', y_train.shape) # 감정 데이터 형태 확인
print('Shape of test input data tensor:', X_test.shape) # 리뷰 데이터의 형태 확인
print('Shape of test label tensor:', y_test.shape) # 감정 데이터 형태 확인

Shape of train input data tensor: (369609, 100)
Shape of train label tensor: (369609,)
Shape of test input data tensor: (46830, 100)
Shape of test label tensor: (46830,)


In [148]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Flatten, Dropout
from tensorflow.keras import layers, models

embedding_dim = 50 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

# model = Sequential()
# model.add(Embedding(len(word_vocab)+1, embedding_dim))
# model.add(Conv1D(256, kernel_size, padding='valid', activation='relu'))
# model.add(Conv1D(256, kernel_size, padding='valid', activation='relu'))
# model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(hidden_units, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))

vocab_size = 30000
word_vector_dim = 64

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(len(word_vocab)+1, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.Conv1D(512, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(3))
model.add(tf.keras.layers.Conv1D(512, 5, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D())
model.add(tf.keras.layers.Conv1D(512, 7, activation='relu'))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dropout(rate=0.5))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics=['accuracy',
                                                                            tf.keras.metrics.TrueNegatives(name='true_negatives'),
                                                                            tf.keras.metrics.TruePositives(name='true_positives'),
                                                                            tf.keras.metrics.FalseNegatives(name='false_negatives'),
                                                                            tf.keras.metrics.FalsePositives(name='false_positives')])
# 이진 분류이므로 손실함수는 binary_crossentropy 사용, 에폭마다 정확도를 보기 위해 accuracy 적용
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, None, 64)          10688     
                                                                 
 conv1d_48 (Conv1D)          (None, None, 512)         98816     
                                                                 
 max_pooling1d_36 (MaxPoolin  (None, None, 512)        0         
 g1D)                                                            
                                                                 
 conv1d_49 (Conv1D)          (None, None, 512)         1311232   
                                                                 
 max_pooling1d_37 (MaxPoolin  (None, None, 512)        0         
 g1D)                                                            
                                                                 
 conv1d_50 (Conv1D)          (None, None, 512)       

In [149]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.2, # new_lr = lr * factor.
    patience=2,
    cooldown=2, # number of epochs to wait before resuming normal operation after lr has been reduced.  
    min_lr=0
)
model.fit(X_train, y_train, epochs= 100, batch_size = 100, validation_split=0.2, callbacks=[early, reduce_lr])

model.save('conv_red8.h5')

Epoch 1/100


2957/2957 [==============================] - 595s 201ms/step - loss: 0.0843 - accuracy: 0.9018 - true_negatives: 266643.0000 - true_positives: 13.0000 - false_negatives: 28948.0000 - false_positives: 83.0000 - val_loss: 0.0750 - val_accuracy: 0.9036 - val_true_negatives: 66797.0000 - val_true_positives: 0.0000e+00 - val_false_negatives: 7125.0000 - val_false_positives: 0.0000e+00 - lr: 0.0010
Epoch 2/100
2957/2957 [==============================] - 598s 202ms/step - loss: 0.0739 - accuracy: 0.9107 - true_negatives: 263790.0000 - true_positives: 5499.0000 - false_negatives: 23462.0000 - false_positives: 2936.0000 - val_loss: 0.0692 - val_accuracy: 0.9183 - val_true_negatives: 66164.0000 - val_true_positives: 1722.0000 - val_false_negatives: 5403.0000 - val_false_positives: 633.0000 - lr: 0.0010
Epoch 3/100
1882/2957 [==================>...........] - ETA: 3:22 - loss: 0.0705 - accuracy: 0.9163 - true_negatives: 167324.0000 - true_positives: 5126.0000 - false_negatives: 13381.0000 - fals

KeyboardInterrupt: 

In [115]:
pred_y_train = model.predict(X_train)
print(model.evaluate(X_train, y_train))

pred_y_test = model.predict(X_test)
print(model.evaluate(X_test, y_test))

  339/13806 [..............................] - ETA: 7:51

KeyboardInterrupt: 

In [16]:
train_org = pd.read_csv(r'data/train_aug.csv')
train_org = train_org.dropna()
test_org = pd.read_csv(r'data/test_aug.csv')
train_org['pred'] = np.where(pred_y_train > 0.5, 1, 0)
test_org['pred'] = np.where(pred_y_test > 0.5, 1, 0)


In [150]:

test = pd.read_csv(r'data/champions.csv')
test_org = pd.read_csv(r'data/champions.csv')
test = test.fillna(0)
test_org = test_org.fillna(0)
test['text'] = test['text'].apply(lambda x : j2hcj(h2j(x)))
test_list = textToList(test['text'])
test_label = test['immoral']
text_sequences_test = tokenizer.texts_to_sequences(test_list)
X_test = pad_sequences(text_sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 100 글자가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_test = np.array(test_label) # 각 리뷰의 감정을 넘파이 배열로 만든다.

# pred_y_test = model2.predict(X_test)
# test_org['pred'] = np.where(pred_y_test > 0.5, 1, 0)
# print(classification_report(test_org['immoral'], test_org['pred']))

NameError: name 'model2' is not defined

In [72]:

test = pd.read_csv(r"C:\Users\llljw\Downloads\clean.txt", sep = 'aaaaaaaaaaaaaaaaaaaaaaaa')
test.columns = ['text']
test['immoral'] = 0
test_org = test.copy()
test = test.fillna(0)
test_org = test_org.fillna(0)
test['text'] = test['text'].apply(lambda x : j2hcj(h2j(x)))
test_list = textToList(test['text'])
test_label = test['immoral']
text_sequences_test = tokenizer.texts_to_sequences(test_list)
X_test = pad_sequences(text_sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 100 글자가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_test = np.array(test_label) # 각 리뷰의 감정을 넘파이 배열로 만든다.

pred_y_test = model2.predict(X_test)
test_org['pred'] = np.where(pred_y_test > 0.5, 1, 0)
print(classification_report(test_org['immoral'], test_org['pred']))

9/9 [==============================] - 0s 875us/step
              precision    recall  f1-score   support

           0       1.00      0.91      0.95       269
           1       0.00      0.00      0.00         0

    accuracy                           0.91       269
   macro avg       0.50      0.46      0.48       269
weighted avg       1.00      0.91      0.95       269



C:\Users\llljw\AppData\Local\Temp\ipykernel_17788\2565546387.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test = pd.read_csv(r"C:\Users\llljw\Downloads\clean.txt", sep = 'aaaaaaaaaaaaaaaaaaaaaaaa')
c:\Users\llljw\anaconda3\envs\mzpark\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\llljw\anaconda3\envs\mzpark\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [19]:
from sklearn.metrics import classification_report
print(classification_report(train_org['immoral'], train_org['pred']))
print(classification_report(test_org['immoral'], test_org['pred']))

              precision    recall  f1-score   support

           0       0.75      0.79      0.77    333523
           1       0.24      0.21      0.23    108258

    accuracy                           0.65    441781
   macro avg       0.50      0.50      0.50    441781
weighted avg       0.63      0.65      0.64    441781

              precision    recall  f1-score   support

           0       0.88      0.95      0.91     41924
           1       0.81      0.65      0.72     14718

    accuracy                           0.87     56642
   macro avg       0.84      0.80      0.82     56642
weighted avg       0.86      0.87      0.86     56642



In [152]:
models = []
modelName = []
# models.append(tf.keras.models.load_model('conv3.h5'))
# modelName.append('conv3.h5')
# models.append(tf.keras.models.load_model('conv_red0.h5'))
# modelName.append('conv_red0.h5')
# models.append(tf.keras.models.load_model('conv_red6.h5'))
# modelName.append('conv_red6.h5')
# models.append(tf.keras.models.load_model('conv_red5.h5'))
# modelName.append('conv_red5.h5')
models.append(tf.keras.models.load_model('version1.h5'))
modelName.append('version1.h5')
models.append(tf.keras.models.load_model('version2.h5'))
modelName.append('version2.h5')
models.append(tf.keras.models.load_model('version3.h5'))
modelName.append('version3.h5')
pred = []
for m, name in zip(models, modelName):
    pred_y_test = m.predict(X_test)
    pred.append(pred_y_test.reshape(-1,))
    pred_y_test = np.where(pred_y_test > 1, 1, 0)
    print(name)
    print(classification_report(y_test, pred_y_test.reshape(-1,)))
    print('*' * 30)

202/202 [==============================] - 8s 32ms/step
version1.h5
              precision    recall  f1-score   support

         0.0       0.73      0.09      0.17      4859
         1.0       0.25      0.89      0.39      1602
         2.0       0.00      0.00      0.00         1
         4.0       0.00      0.00      0.00         1

    accuracy                           0.29      6463
   macro avg       0.24      0.25      0.14      6463
weighted avg       0.61      0.29      0.22      6463

******************************


c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


202/202 [==============================] - 14s 61ms/step
version2.h5
              precision    recall  f1-score   support

         0.0       0.74      0.38      0.50      4859
         1.0       0.24      0.61      0.35      1602
         2.0       0.00      0.00      0.00         1
         4.0       0.00      0.00      0.00         1

    accuracy                           0.43      6463
   macro avg       0.25      0.25      0.21      6463
weighted avg       0.62      0.43      0.46      6463

******************************


c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


202/202 [==============================] - 8s 33ms/step
version3.h5
              precision    recall  f1-score   support

         0.0       0.76      0.45      0.57      4859
         1.0       0.26      0.58      0.36      1602
         2.0       0.00      0.00      0.00         1
         4.0       0.00      0.00      0.00         1

    accuracy                           0.48      6463
   macro avg       0.26      0.26      0.23      6463
weighted avg       0.64      0.48      0.52      6463

******************************


c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [90]:
pred_b = []
for p in pred:
    pred_b.append(np.where(p > 0.5, 1, 0))
pred_sum = np.sum(pred_b, axis = 0)

In [153]:
pred_b = []
for p in pred:
    mn = np.where(p > 0.5, 1, 0)
    print(classification_report(y_test, mn))


              precision    recall  f1-score   support

         0.0       0.77      0.04      0.07      4859
         1.0       0.25      0.97      0.40      1602
         2.0       0.00      0.00      0.00         1
         4.0       0.00      0.00      0.00         1

    accuracy                           0.27      6463
   macro avg       0.25      0.25      0.12      6463
weighted avg       0.64      0.27      0.15      6463

              precision    recall  f1-score   support

         0.0       0.73      0.26      0.39      4859
         1.0       0.24      0.71      0.36      1602
         2.0       0.00      0.00      0.00         1
         4.0       0.00      0.00      0.00         1

    accuracy                           0.37      6463
   macro avg       0.24      0.24      0.19      6463
weighted avg       0.61      0.37      0.38      6463

              precision    recall  f1-score   support

         0.0       0.77      0.29      0.42      4859
         1.0       0.

c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\mz

In [160]:
test.loc[test['immoral'] > 1, 'immoral'] = 1

In [159]:
df = pd.DataFrame(pred)
pred_f = df.mean(axis = 0).values
mn = np.where(pred_f > 0.5, 1, 0)
print(classification_report(y_test, mn))

              precision    recall  f1-score   support

         0.0       0.74      0.09      0.16      4859
         1.0       0.25      0.90      0.39      1602
         2.0       0.00      0.00      0.00         1
         4.0       0.00      0.00      0.00         1

    accuracy                           0.29      6463
   macro avg       0.25      0.25      0.14      6463
weighted avg       0.62      0.29      0.22      6463



c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\mzp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [96]:

test_org = pd.read_csv(r'data/champions.csv')
test_org['pred_sum'] = pred_sum
test_org['pred_f'] = pred_f
test_org
test_org[['pred_f', 'pred_sum', 'immoral', 'text']].to_csv('cmp_check.csv')

In [81]:
len(test_org)

6463

In [61]:
test_org[test_org['pred_f'] > 0.8][['pred_f', 'pred_sum', 'immoral', 'text']].to_csv('over8.csv')